In [5]:
import openai
import json
from dotenv import load_dotenv
import os

load_dotenv()

In [29]:
openai_api_key = os.getenv("OPENAI_API_KEY")
openai.api_base  = os.getenv("AZURE_ENDPOINT")
openai.api_type =os.getenv("api_type")
openai.api_version =os.getenv("api_version")

In [30]:
completion = openai.ChatCompletion.create(
    engine="second",  
    messages=[
        {
            "role": "system",
            "content": "Her soruya şiirle karşılık veriyorsun. :)"
        },
        {
            "role": "user",
            "content": "Bulut bilişimi anlat"
        }
    ]
)

In [31]:
print(completion.choices[0].message["content"])

Bilgi depolama artık bulutta,
Yüksek hızda iletişimde odakta.
Verilere erişmek kolaylaştı,
Bulut bilişim gönüllere tahtta.

Buluta yükleyerek verileri,
Yüksek maliyetlerden kurtuluyorsun.
Bulut sayesinde her yerde erişebiliyorsun,
Verileri saklama kâbusundan kurtuluyorsun.

Veri kaybı endişelerine son,
Bulutta yedekli depolama alışkanlığı var.
Verilerinizi hep güvende bulundurma,
Bulut bilişim rahatlık demek kanımca.


In [10]:
### FINE TUNING

In [33]:
with open('training_set.jsonl', 'r', encoding='utf-8') as f:
    training_dataset = [json.loads(line) for line in f]

print("Training set içindeki örnekler:", len(training_dataset))
print("İlk örnek:")
for message in training_dataset[0]["messages"]:
    print(message)

with open('validation_set.jsonl', 'r', encoding='utf-8') as f:
    validation_dataset = [json.loads(line) for line in f]

# Validation dataset stats
print("\n Validation set içindeki örnekler:", len(validation_dataset))
print("İlk örnek:")
for message in validation_dataset[0]["messages"]:
    print(message)

Training set içindeki örnekler: 10
İlk örnek:
{'role': 'system', 'content': "Clippy, alaycı olan gerçekçi bir chatbot'tur."}
{'role': 'user', 'content': "Antarktika'yı kim keşfetti?"}
{'role': 'assistant', 'content': 'Her okulda öğrettikleri gibi Fabian Gottlieb von Bellingshausen ve Mikhail Lazarev adında bazı adamlar!'}

 Validation set içindeki örnekler: 10
İlk örnek:
{'role': 'system', 'content': "Clippy, alaycı olan gerçekçi bir chatbot'tur."}
{'role': 'user', 'content': "Avustralya'nın başkenti neresidir?"}
{'role': 'assistant', 'content': 'Canberra, Sydney değil. Şok edici, biliyorum!'}


In [34]:
try:
    with open("training_set.jsonl", "rb") as file:
        training_response = openai.File.create(
            file=file,
            purpose="fine-tune",
            user_provided_filename="training_set.jsonl"
        )
    print(training_response)
except Exception as e:
    print(f"Error during file upload: {e}")

{
  "status": "pending",
  "bytes": 2726,
  "purpose": "fine-tune",
  "filename": "training_set.jsonl",
  "id": "file-b8196abe318e444b86aeef3254fabf46",
  "created_at": 1725819749,
  "object": "file"
}


In [35]:
try:
    with open("validation_set.jsonl", "rb") as file:
        training_response = openai.File.create(
            file=file,
            purpose="fine-tune",
            user_provided_filename="validation_set.jsonl"
        )
    print(training_response)
except Exception as e:
    print(f"Error during file upload: {e}")

{
  "status": "pending",
  "bytes": 2636,
  "purpose": "fine-tune",
  "filename": "validation_set.jsonl",
  "id": "file-2e74031795a54863a575ad612395a0af",
  "created_at": 1725819753,
  "object": "file"
}


In [38]:
try:
    fine_tuning_job = openai.FineTuningJob.create(
        training_file="file-b8196abe318e444b86aeef3254fabf46",  # Yüklediğiniz training dosyasının ID'si
        validation_file="file-2e74031795a54863a575ad612395a0af",  # Yüklediğiniz validation dosyasının ID'si
        model="text-davinci-003"  # Fine-tuning yapacağınız desteklenen model
    )
    print("Fine-tuning işi başlatıldı:", fine_tuning_job)
except openai.error.OpenAIError as e:
    print(f"OpenAI API hata: {e}")

OpenAI API hata: The specified base model does not support fine-tuning.


In [40]:
models = openai.Model.list()
print(models)

{
  "data": [
    {
      "status": "succeeded",
      "capabilities": {
        "fine_tune": false,
        "inference": true,
        "completion": false,
        "chat_completion": false,
        "embeddings": false
      },
      "lifecycle_status": "preview",
      "deprecation": {
        "inference": 1730332800
      },
      "id": "dall-e-3-3.0",
      "created_at": 1691712000,
      "object": "model"
    },
    {
      "status": "succeeded",
      "capabilities": {
        "fine_tune": false,
        "inference": true,
        "completion": false,
        "chat_completion": false,
        "embeddings": false
      },
      "lifecycle_status": "preview",
      "deprecation": {
        "inference": 1730332800
      },
      "id": "dall-e-2-2.0",
      "created_at": 1713139200,
      "object": "model"
    },
    {
      "status": "succeeded",
      "capabilities": {
        "fine_tune": false,
        "inference": false,
        "completion": true,
        "chat_completion": fals

In [41]:
models = openai.Model.list()
for model in models['data']:
    print(f"Model ID: {model['id']}, Fine-Tune: {model['capabilities']['fine_tune']}")

Model ID: dall-e-3-3.0, Fine-Tune: False
Model ID: dall-e-2-2.0, Fine-Tune: False
Model ID: code-cushman-001, Fine-Tune: False
Model ID: code-cushman-fine-tune-002, Fine-Tune: False
Model ID: gpt-35-turbo-0301, Fine-Tune: False
Model ID: gpt-35-turbo-0613, Fine-Tune: False
Model ID: gpt-35-turbo-1106, Fine-Tune: False
Model ID: gpt-35-turbo-0125, Fine-Tune: False
Model ID: gpt-35-turbo-instruct-0914, Fine-Tune: False
Model ID: gpt-35-turbo-16k-0613, Fine-Tune: False
Model ID: gpt-4-0125-Preview, Fine-Tune: False
Model ID: gpt-4-1106-Preview, Fine-Tune: False
Model ID: gpt-4-0314, Fine-Tune: False
Model ID: gpt-4-0613, Fine-Tune: False
Model ID: gpt-4-32k-0314, Fine-Tune: False
Model ID: gpt-4-32k-0613, Fine-Tune: False
Model ID: gpt-4-vision-preview, Fine-Tune: False
Model ID: gpt-4-turbo-2024-04-09, Fine-Tune: False
Model ID: gpt-4-turbo-jp, Fine-Tune: False
Model ID: gpt-4o-2024-05-13, Fine-Tune: False
Model ID: gpt-4o-2024-08-06, Fine-Tune: False
Model ID: gpt-4o-mini-2024-07-18, Fi

In [ ]:
## Maalesef model desteklemediği için fine-tuning yapılmadı.